# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Импорт библиотек

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.dummy import DummyClassifier

import scipy.stats as stats

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Откройте и изучите файл

**Описание данных**

Признаки:
* calls — количество звонков
* minutes — суммарная длительность звонков в минутах
* messages — количество sms-сообщений
* mb_used — израсходованный интернет-трафик в Мб

Целевой признак:
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [3]:
df = pd.read_csv('users_behavior.csv')
print(df.info())
display(df.describe().transpose())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Пропусков нет, неожиданных значений нет.

# Разбейте данные на выборки

In [4]:
features = df.iloc[:, :-1]
target = df.iloc[:, -1]
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=.25, random_state=24)

Данные разбиты на 2 выборки обучающую и тестовую. Размеры которых соотносятся как 3 : 1 соответсвенно.

# Исследуйте модели

Для сравнительно быстрого поиска лучших гиперпараметров моделей воспользуемся случайным поиском гиперпараметров **RandomizedSearchCV**. Количество итераций поиска установим на 100.

## Дерево классификации

In [5]:
parameters_tree = {
    'max_depth' : range(1, 21),
    'min_samples_split' : range(2, 51),
    'max_leaf_nodes' : range(2, 10)    
}
random_search = RandomizedSearchCV(DecisionTreeClassifier(random_state=24), param_distributions=parameters_tree, n_iter=100, random_state=24)
random_search.fit(features_train, target_train)
print(f'''Параметры лучшего дерева: 
глубина дерева = {random_search.best_params_['max_depth']}, 
доля примеров из общей выборки в одном узле = {random_search.best_params_['min_samples_split']}, 
количество листьев = {random_search.best_params_['max_leaf_nodes']},
точность модели при кросс-валидации = {random_search.best_score_}.''')

Параметры лучшего дерева: 
глубина дерева = 18, 
доля примеров из общей выборки в одном узле = 45, 
количество листьев = 9,
точность модели при кросс-валидации = 0.8012448132780083.


Создадим модель с полученными гиперпараметрами и обучим её.

In [6]:
best_tree = DecisionTreeClassifier(max_depth=18, min_samples_split=45, max_leaf_nodes=9, random_state=24)
best_tree.fit(features_train, target_train)
print('Дерево обучено')

Дерево обучено


## Случайный лес

In [7]:
parameters_forest = {
    'n_estimators' : range(2, 51),
    'max_depth' : range(1, 21),
    'min_samples_split' : range(2, 51),
    'max_leaf_nodes' : range(2, 10)    
}
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=24), param_distributions=parameters_forest, n_iter=100, random_state=24)
random_search.fit(features_train, target_train)
print(f'''Параметры лучшего леса: 
количество деревьев = {random_search.best_params_['n_estimators']}
глубина дерева = {random_search.best_params_['max_depth']}, 
доля примеров из общей выборки в одном узле = {random_search.best_params_['min_samples_split']}, 
количество листьев = {random_search.best_params_['max_leaf_nodes']},
точность модели при кросс-валидации = {random_search.best_score_}.''')

Параметры лучшего леса: 
количество деревьев = 10
глубина дерева = 17, 
доля примеров из общей выборки в одном узле = 3, 
количество листьев = 9,
точность модели при кросс-валидации = 0.8074688796680498.


Создадим модель с полученными гиперпараметрами и обучим её.

In [8]:
best_forest = RandomForestClassifier(n_estimators=10 ,max_depth=17, min_samples_split=3, max_leaf_nodes=9, random_state=24)
best_forest.fit(features_train, target_train)
print('Лес обучен')

Лес обучен


## Логистическая регрессия

In [9]:
parameters_logistic = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : stats.loguniform(10**-8, 10),
    'l1_ratio' : stats.uniform(0, 1), 
}
random_search = RandomizedSearchCV(LogisticRegression(random_state=24), param_distributions=parameters_logistic, n_iter=100, random_state=24)
random_search.fit(features_train, target_train)
print(f'''Параметры лучшей регрессии: 
штрафование = {random_search.best_params_['penalty']}
C = {random_search.best_params_['C']}, 
l1_ratio = {random_search.best_params_['l1_ratio']}, 
точность модели при кросс-валидации = {random_search.best_score_}.''')

Параметры лучшей регрессии: 
штрафование = l2
C = 7.667849003365796e-06, 
l1_ratio = 0.3664147530835151, 
точность модели при кросс-валидации = 0.7489626556016598.


In [10]:
model_regression = LogisticRegression(penalty='l2', C=7.667849003365796e-06, l1_ratio=0.3664147530835151, random_state=24)
model_regression.fit(features_train, target_train)
print('Логистичесая регрессия обучена')

Логистичесая регрессия обучена


# Проверьте модель на тестовой выборке

**Дерево классификации**

In [11]:
accuracy = best_tree.score(features_test, target_test)
accuracy

0.7947761194029851

**Случайный лес**

In [12]:
accuracy = best_forest.score(features_test, target_test)
accuracy

0.7960199004975125

**Логистическая регрессия**

In [13]:
accuracy = model_regression.score(features_test, target_test)
accuracy

0.7251243781094527

# (бонус) Проверьте модели на адекватность

Для этого сгенерируем случайную модель, и проверим её на точность.

In [14]:
dummy = DummyClassifier(strategy='most_frequent', random_state=24)
dummy.fit(features_train, target_train)
dummy.score(features_test, target_test)

0.6840796019900498

Что меньше чем у любой из обученных моделей.

# Вывод

Лучшей моделью для данной задачи оказался случайный лес. Он дал наибольшую точность предсказания среди других моделей (79.6 % точности). Недалеко позади находиться одиночное дерево (79.5 %). И на последнем месте логистическая регрессия (72.5 %).

При этом все модели успешно прошли проверку на адекватность.